In [2]:
%tensorflow_version 2.0.0

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.0.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [4]:
from platform import python_version

print(python_version())

3.6.9


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = 'gdrive/My Drive/kaggle/google-quest-challenge'

#data_dir = '/home/jupyter/google_quest_datasets/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import subprocess
import sys

reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

if 'sacremoses' not in installed_packages:
   !pip install sacremoses >/dev/null

if 'transformers' not in installed_packages:
   !pip install transformers >/dev/null
else:
   import transformers
   print(transformers.__version__)


In [0]:
#!pip install iterative-stratification 

In [8]:
# coding: utf-8

import os 
import transformers, sys, os, gc
import numpy as np, pandas as pd, math
import torch, random, os, multiprocessing, glob
import torch.nn.functional as F
import torch, time

#sys.path.insert(0, "gdrive/My Drive/kaggle/google-quest-challenge/")
sys.path.insert(0, data_dir)

from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from scipy.stats import spearmanr
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from transformers import (RobertaTokenizer, RobertaModel, BertPreTrainedModel, 
                          RobertaForSequenceClassification, RobertaConfig, AdamW, 
                          get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP )
#WEIGHTS_NAME, CONFIG_NAME, 

from os.path import join as path_join

from tqdm import tqdm
import logging

logging.basicConfig(filename=path_join('gdrive','My Drive','kaggle','submissions','roberta_seed_1029','robertamodel1029.log') ,level=logging.WARNING)
    
print(transformers.__version__)

def set_seeds(SEED=42):
  os.environ["PYTHONHASHSEED"] = str(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

SEED=1029
set_seeds(SEED)
HEAD_TAIL = True

2.4.1


In [0]:
#logging.basicConfig(filename=path_join('gdrive','My Drive','kaggle','submissions','roberta_seed42','robertamodel.log') ,level=logging.WARNING)
logging.warning('Roberta log')
logging.warning('Roberta log - seed 1029')

### Create USE Features 

In [17]:
import os
import re
import gc
import pickle  
import random
import keras

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
#import keras.backend as K

#from keras.models import Model
#from keras.layers import Dense, Input, Dropout, Lambda
#from keras.optimizers import Adam
#from keras.callbacks import Callback
from scipy.stats import spearmanr, rankdata
from os.path import join as path_join
from numpy.random import seed
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet
import re

seed(SEED)
#tf.random.set_seed(SEED)
random.seed(SEED)

Using TensorFlow backend.


In [18]:
train = pd.read_csv(path_join(data_dir, 'train.csv'))
test = pd.read_csv(path_join(data_dir, 'test.csv'))
print(train.shape, test.shape)

(6079, 41) (476, 11)


In [19]:
train.answer_well_written.value_counts()

1.000000    2634
0.888889    2175
0.777778     727
0.666667     274
0.833333     211
0.555556      38
0.444444       8
0.500000       8
0.333333       4
Name: answer_well_written, dtype: int64

In [20]:
train[['answer','answer_well_written']]

,answer,answer_well_written
0,"I just got extension tubes, so here's the skin...",1.000000
1,It might be helpful to look into the definitio...,0.888889
2,Do you even need grooves? We make several pro...,0.888889
3,"Sending an ""affidavit"" it is a dispute between...",1.000000
4,Check out Image Trace in Adobe Illustrator. \n...,1.000000
...,...,...
6074,If you're thinking about wearing a ski helmet ...,0.888889
6075,\nYou can replace the pads (as stated elsewher...,0.888889
6076,Maybe help if can be fixes origin of this erro...,0.555556
6077,"As a non-mathematician, I am somewhat mystifie...",1.000000


## Features 

In [0]:
tmp = pd.DataFrame([['unknown1', 'LIFE_ARTS'],['unknown2', 'CULTURE'],['unknown3', 'SCIENCE'],
             ['unknown4', 'STACKOVERFLOW'],['unknown5', 'TECHNOLOGY'],['unknown6', 'LIFE_ARTS'],
              ['unknown7', 'CULTURE'],['unknown8', 'SCIENCE'],['unknown9', 'STACKOVERFLOW'],
              ['unknown10', 'TECHNOLOGY'],['unknown11', 'CULTURE'],['unknown12', 'SCIENCE'],
              ['unknown13', 'STACKOVERFLOW'],['unknown14', 'TECHNOLOGY'],['unknown15', 'LIFE_ARTS'],
              ['unknown16', 'LIFE_ARTS'],['unknown17', 'CULTURE'],['unknown18', 'SCIENCE'],
             ['unknown19', 'STACKOVERFLOW'],['unknown20', 'TECHNOLOGY'],['unknown21', 'LIFE_ARTS'],
              ['unknown22', 'CULTURE'],['unknown23', 'SCIENCE'],['unknown24', 'STACKOVERFLOW'],
              ['unknown25', 'TECHNOLOGY'],['unknown26', 'CULTURE'],['unknown27', 'SCIENCE'],
              ['unknown28', 'STACKOVERFLOW'],['unknown29', 'TECHNOLOGY'],['unknown30', 'LIFE_ARTS'] ,
                    ['unknown31', 'LIFE_ARTS'],['unknown32', 'CULTURE'],['unknown33', 'SCIENCE'],
             ['unknown34', 'STACKOVERFLOW'],['unknown35', 'TECHNOLOGY'],['unknown36', 'LIFE_ARTS'],
              ['unknown37', 'CULTURE'],['unknown38', 'SCIENCE'],['unknown39', 'STACKOVERFLOW'],
              ['unknown40', 'TECHNOLOGY']
             ],columns =  ['netloc', 'category'])

In [22]:
find = re.compile(r"^[^.]*")

train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

features = ['netloc', 'category']
merged = pd.concat([train[features], test[features]])

#Adding unknown url
merged = merged.append(tmp)

ohe = OneHotEncoder()
ohe.fit(merged)

features_train = ohe.transform(train[features]).toarray()
features_test = ohe.transform(test[features]).toarray()
print(features_train.shape, features_test.shape)
#Need to add 40 more unknown categories for private test set !! 

(6079, 104) (476, 104)


In [0]:
module_url = path_join(data_dir,'4/') #path_join(data_dir,'use/')
embed = hub.load(module_url)

In [0]:
input_columns = ['question_title', 'question_body', 'answer']

In [25]:
embeddings_train = {}
embeddings_test = {}
for text in input_columns:
    print(text)
    train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()
    
    curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
    ind = 0
    while ind*batch_size < len(train_text):
        curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1
        
    ind = 0
    while ind*batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1    
        
    embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)
    
del embed
gc.collect()

question_title
question_body
answer


44

In [26]:
#512 columns for each row, each column i.e. Row 1 -> question_title -> 512 features
len(embeddings_train), len(embeddings_test), embeddings_test['question_title_embedding'].shape

(3, 3, (476, 512))

### Distance features

In [0]:
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)

dist_features_train = np.array([
    l2_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    l2_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
    cos_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
    cos_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding'])
]).T

dist_features_test = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding'])
]).T

X_train = np.hstack([item for k, item in embeddings_train.items()] + [features_train, dist_features_train])
X_test = np.hstack([item for k, item in embeddings_test.items()] + [features_test, dist_features_test])
#y_train = train[targets].values

In [28]:
X_train.shape, X_test.shape

((6079, 1646), (476, 1646))

In [0]:
# import pickle
# with open("X_train.pickle", "rb") as input_file:
#       X_train = pickle.load(input_file)
 

In [29]:
X_train.shape,

((6079, 1646),)

In [30]:
!nvidia-smi

Sat Feb  8 14:38:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    39W / 250W |   2861MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [31]:
from numba import cuda
cuda.select_device(0)
cuda.close() 

cuda.select_device(0)
cuda.close()
cuda.select_device(0)


<weakproxy at 0x7f92b69cdb38 to Device at 0x7f9256ff6e10>

In [32]:
!nvidia-smi

Sat Feb  8 14:38:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    38W / 250W |    263MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Model

In [0]:
LEN_USE_FEATURES = 1646 #1606 


class RobertaClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        #self.out_proj = nn.Linear(config.hidden_size + LEN_USE_FEATURES, config.num_labels)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    #def forward(self, features, pooled_out , hidden_st, usefeatures, **kwargs):
    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        #x = features[:, 0, :].reshape((-1, 1, 768))


        # h12 = hidden_st[-1][:, 0].reshape((-1, 1, 768))
        # h11 = hidden_st[-2][:, 0].reshape((-1, 1, 768))
        # h10 = hidden_st[-3][:, 0].reshape((-1, 1, 768))
        # h9  = hidden_st[-4][:, 0].reshape((-1, 1, 768))
        # all_h = torch.cat([ h9, h10, h11, h12], 1)
        # all_h_mean = torch.mean(all_h, 1).reshape((-1,1,768))
        # x = torch.mean(torch.cat([x,all_h_mean],1),1)


        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        #x = self.out_proj(torch.cat([x,usefeatures],1))


        return x

class RobertaForSequenceClassification_v2(BertPreTrainedModel):
    config_class = RobertaConfig
    pretrained_model_archive_map = ROBERTA_PRETRAINED_MODEL_ARCHIVE_MAP
    base_model_prefix = "roberta"

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)

    # def forward(
    #     self,
    #     input_ids=None,
    #     attention_mask=None,
    #     token_type_ids=None,
    #     position_ids=None,
    #     head_mask=None,
    #     inputs_embeds=None,
    #     labels=None,
    # ):

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        usefeatures = None
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.RobertaConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import RobertaTokenizer, RobertaForSequenceClassification
        import torch

        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForSequenceClassification.from_pretrained('roberta-base')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

        """
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        sequence_output = outputs[0]

        pooled_output = outputs[1] 
        hidden_states = outputs[2]
        
        #logits = self.classifier(sequence_output, pooled_output, hidden_states, usefeatures)
        logits = self.classifier(sequence_output) 

        outputs = (logits,) + outputs[2:]
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)



In [0]:

train = pd.read_csv( path_join(data_dir,"train.csv"))
test = pd.read_csv( path_join(data_dir, "test.csv"))

target_cols_questions = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written'] 

target_cols_answers =  ['answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

In [0]:
weights_questions = np.ones((len(train),))
mask = np.sum(train[target_cols_questions]>0.5,1)
weights_questions = weights_questions + mask 
loss_weight_questions = 1.0 / weights_questions.mean()

train['weights_questions'] = weights_questions
target_cols_questions = target_cols_questions + ['weights_questions']




weights_answers = np.ones((len(train),))
mask = np.sum(train[target_cols_answers]>0.5,1)
weights_answers = weights_answers + mask 
loss_weight_answers = 1.0 / weights_answers.mean()

train['weights_answers'] = weights_answers
target_cols_answers = target_cols_answers + ['weights_answers']




In [0]:
# helpers
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "</s>":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

### Tokenize Answers

In [0]:
def _trim_input_answers(title, question, answer, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239):
    
    #293+239+30 = 508 + 4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        # if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
        #     raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        # t = t[:t_new_len]
        # q = q[:q_new_len]
        # a = a[:a_new_len]


        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        q_len_head = round(q_new_len/2)
        q_len_tail = -1* (q_new_len -q_len_head)
        a_len_head = round(a_new_len/2)
        a_len_tail = -1* (a_new_len -a_len_head)        ## Head+Tail method .
        t = t[:t_new_len]
        if HEAD_TAIL :
            q = q[:q_len_head]+q[q_len_tail:]
            a = a[:a_len_head]+a[a_len_tail:]
        else:
            q = q[:q_new_len]
            a = a[:a_new_len] ## No Head+Tail ,usual processing
    
    return t, q, a



def _convert_to_bert_inputs_answers(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken =  ['<s> '] + title + ['</s> </s> '] + question + ['</s> </s> '] + answer + ['</s>'] 


    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]



def compute_input_arays_answers(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q, a  = instance.question_title,instance.question_body, instance.answer
        t, q, a  = _trim_input_answers(t, q, a, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs_answers(t,q, a,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]



### Tokenize questions

In [0]:
def _trim_input(title, question, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=479):
    
    #3 + 479 + 30
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    
    t_len = len(t)
    q_len = len(q)
    
    if (t_len+q_len+3) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
            q_new_len = q_max_len
        else:
            t_new_len = t_max_len
            q_new_len = q_max_len
      
        if q_max_len > q_len:
            q_new_len = q_len
            
     
        if t_new_len+q_new_len+3 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + q_new_len + 3)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        
    return t, q

def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ['<s>'] + title + ['</s> </s> '] + question + ['</s>'] 
    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q  = instance.question_title, instance.question_body
        t, q  = _trim_input(t, q, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]




In [0]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, usefeatures, lengths, labels = None):
        
        self.inputs = inputs
        self.usefeatures = usefeatures
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths
       

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        usef            = self.usefeatures[idx,:]
        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments,  usef, labels, lengths
        return input_ids, input_masks, input_segments, usef, lengths

    def __len__(self):
        return len(self.inputs[0])


In [0]:
def get_loss_fn_c_answer(criterion, lgits, lbls):
    #return (criterion(lgits,lbls[:,0:30]) * lbls[:,30].view(-1,1)).mean()
    return (criterion(lgits,lbls[:,0:9]) * lbls[:,9].view(-1,1)).mean()




def get_loss_fn_c(criterion, lgits, lbls):
    #print(lgits.shape, lbls.shape,lbls[:,0:30].shape)
    #print(criterion(lgits,lbls[:,0:30]).shape, lbls[:,30].shape )
    return (criterion(lgits,lbls[:,0:21]) * lbls[:,21].view(-1,1)).mean()



In [0]:
def train_model_answers(train_loader, optimizer, criterion, scheduler):
    
    model_a.train()
    avg_loss = 0.
    tk0 = tqdm(enumerate(train_loader),position=0, leave=True )
    
    for idx, batch in tk0:
        
        input_ids, input_masks, input_segments, usefeatures, labels, _ = batch
        input_ids, input_masks, input_segments, usefeatures, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), usefeatures.to(device) ,labels.to(device)            
         
        output_train = model_a(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures
                            )
        logits = output_train[0] #output preds

        loss = get_loss_fn_c_answer(criterion,logits,labels)
        #loss = get_loss_fn_b(criterion,logits,labels)
        #loss = loss / accumulation_steps    #https://gist.github.com/thomwolf/ac7a7da6b1888c2eeac8ac8b9b05d3d3
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del input_ids, input_masks, input_segments, labels, usefeatures

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

In [0]:
def train_model_questions(train_loader, optimizer, criterion, scheduler):
    
    model_q.train()
    avg_loss = 0.
    tk0 = tqdm(enumerate(train_loader),position=0, leave=True )
    
    for idx, batch in tk0:
        
        input_ids, input_masks, input_segments, usefeatures, labels, _ = batch
        input_ids, input_masks, input_segments, usefeatures, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device),usefeatures.to(device), labels.to(device)            
        output_train = model_q(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures
                            )
        logits = output_train[0] #output preds

        loss = get_loss_fn_c(criterion,logits,labels)
        #loss = get_loss_fn_b(criterion,logits,labels)
        #loss = loss / accumulation_steps    #https://gist.github.com/thomwolf/ac7a7da6b1888c2eeac8ac8b9b05d3d3
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del input_ids, input_masks, input_segments, labels, usefeatures

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

In [0]:
def val_model_answers(val_loader, val_shape, batch_size=8):  
    
    avg_val_loss = 0.
    model_a.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, 9))
    original = np.zeros((val_shape, 9))
    
    tk0 = tqdm(enumerate(val_loader),position=0, leave=True )
    with torch.no_grad():
        
        for idx, batch in tk0:
            input_ids, input_masks, input_segments,usefeatures, labels, _ = batch
            input_ids, input_masks, input_segments, usefeatures, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), usefeatures.to(device), labels.to(device)            
            
            output_val = model_a(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures
                            )
            logits = output_val[0] #output preds
            
            vloss = get_loss_fn_c_answer(criterion,logits,labels)
            avg_val_loss += vloss.item() / len(val_loader)


            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            #original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels[:,0:9].detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()

        #Fix 'question_type_spelling' #19, Non Culture = 1
        
        # np.save("preds.npy", preds)
        # np.save("actuals.npy", original)
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(9):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    return avg_val_loss, score/9. , preds

In [0]:
def val_model_questions(val_loader, val_shape, batch_size=8):  
    
    avg_val_loss = 0.
    model_q.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, 21))
    original = np.zeros((val_shape, 21))
    
    tk0 = tqdm(enumerate(val_loader),position=0, leave=True )
    with torch.no_grad():
        
        for idx, batch in tk0:
            input_ids, input_masks, input_segments, usefeatures, labels, _ = batch
            input_ids, input_masks, input_segments, usefeatures, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device),usefeatures.to(device), labels.to(device)            
            
            output_val = model_q(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures
                            )
            logits = output_val[0] #output preds
            
            vloss = get_loss_fn_c(criterion,logits,labels)
            avg_val_loss += vloss.item() / len(val_loader)


            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            #original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels[:,0:21].detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()

        #Fix 'question_type_spelling' #19, Non Culture = 1
        
        # np.save("preds.npy", preds)
        # np.save("actuals.npy", original)
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(21):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    return avg_val_loss, score/21, preds #return 21 questions columns



In [45]:
!nvidia-smi


Sat Feb  8 14:38:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    38W / 250W |    263MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [46]:

set_seeds(SEED)
question_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
input_categories_q = list(train.columns[[1,2]]); 

robert_config_question =  RobertaConfig.from_pretrained('roberta-base')
robert_config_question.num_labels = 21
robert_config_question.output_hidden_states = True 

answer_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
input_categories_a = list(train.columns[[1,2,5]]); 

robert_config_answer =  RobertaConfig.from_pretrained('roberta-base')
robert_config_answer.num_labels = 9
robert_config_answer.output_hidden_states = True 

device = 'cuda' if torch.cuda.is_available() else 'cpu'


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


# In[8]:

NUM_FOLDS = 5  # change this
BATCH_SIZE = 8
epochs = 5 #4     # change this
ACCUM_STEPS = 1
test_predictions = [] 
all_rhos = []

kf = MultilabelStratifiedKFold(n_splits = NUM_FOLDS, random_state = SEED)

y_train_q = train[target_cols_questions].values # dummy
y_train_a = train[target_cols_answers].values # dummy


print(bcolors.FAIL, f"For Every Fold, Train {epochs} Epochs", bcolors.ENDC)

set_seeds(SEED)




 For Every Fold, Train 5 Epochs 


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
xtrain_usefeatures = X_train

In [48]:
best_rhos = []
for fold, (train_index, val_index) in enumerate(kf.split(train.values, y_train_q)):

  #if fold > 1:  
    print(bcolors.HEADER, "Current Fold:", fold, bcolors.ENDC)
    train_df, val_df = train.iloc[train_index], train.iloc[val_index]
    print("Train and Valid Shapes are", train_df.shape, val_df.shape)   
    print(bcolors.HEADER, "Preparing train datasets....", bcolors.ENDC)

    #questions
    inputs_train_q = compute_input_arays(train_df, input_categories_q, question_tokenizer, max_sequence_length=512)
    outputs_train_q = compute_output_arrays(train_df, columns = target_cols_questions)
    outputs_train_q = torch.tensor(outputs_train_q, dtype=torch.float32)
    lengths_train_q = np.argmax(inputs_train_q[0] == 0, axis=1)
    lengths_train_q[lengths_train_q == 0] = inputs_train_q[0].shape[1]

    #answers
    inputs_train_a = compute_input_arays_answers(train_df, input_categories_a, answer_tokenizer, max_sequence_length=512)
    outputs_train_a = compute_output_arrays(train_df, columns = target_cols_answers)
    outputs_train_a = torch.tensor(outputs_train_a, dtype=torch.float32)
    lengths_train_a = np.argmax(inputs_train_a[0] == 0, axis=1)
    lengths_train_a[lengths_train_a == 0] = inputs_train_a[0].shape[1]

    print(bcolors.HEADER, "Preparing Valid datasets....", bcolors.ENDC)
    #questions
    inputs_valid_q = compute_input_arays(val_df, input_categories_q, question_tokenizer, max_sequence_length=512)
    outputs_valid_q = compute_output_arrays(val_df, columns = target_cols_questions)
    outputs_valid_q = torch.tensor(outputs_valid_q, dtype=torch.float32)
    lengths_valid_q = np.argmax(inputs_valid_q[0] == 0, axis=1)
    lengths_valid_q[lengths_valid_q == 0] = inputs_valid_q[0].shape[1]

    #answers
    inputs_valid_a = compute_input_arays_answers(val_df, input_categories_a, answer_tokenizer, max_sequence_length=512)
    outputs_valid_a = compute_output_arrays(val_df, columns = target_cols_answers)
    outputs_valid_a = torch.tensor(outputs_valid_a, dtype=torch.float32)
    lengths_valid_a = np.argmax(inputs_valid_a[0] == 0, axis=1)
    lengths_valid_a[lengths_valid_a == 0] = inputs_valid_a[0].shape[1]


    print(bcolors.HEADER, "Preparing Dataloaders Datasets....", bcolors.ENDC)
    #questions
    train_set_q    = QuestDataset(inputs=inputs_train_q, usefeatures=torch.FloatTensor(xtrain_usefeatures[train_index,:]), lengths=lengths_train_q, labels=outputs_train_q)
    train_loader_q = DataLoader(train_set_q, batch_size=BATCH_SIZE, shuffle=True)
    #answers
    train_set_a    = QuestDataset(inputs=inputs_train_a,usefeatures=torch.FloatTensor(xtrain_usefeatures[train_index,:]), lengths=lengths_train_a, labels=outputs_train_a)
    train_loader_a = DataLoader(train_set_a, batch_size=BATCH_SIZE, shuffle=True)
    #questions
    valid_set_q    = QuestDataset(inputs=inputs_valid_q,usefeatures=torch.FloatTensor(xtrain_usefeatures[val_index,:]), lengths=lengths_valid_q, labels=outputs_valid_q)
    valid_loader_q = DataLoader(valid_set_q, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
    #answers
    valid_set_a    = QuestDataset(inputs=inputs_valid_a, usefeatures=torch.FloatTensor(xtrain_usefeatures[val_index,:]),lengths=lengths_valid_a, labels=outputs_valid_a)
    valid_loader_a = DataLoader(valid_set_a, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

    i = 0
    best_avg_loss_q,best_avg_loss_a    = 100.0 , 100.0
    best_score_q, best_score_a      = -1. , -1.
    best_param_loss_q, best_param_loss_a = None,None
    best_param_score_q,best_param_score_a = None,None

    #model for questions
    #model_q = RobertaForSequenceClassification_v2.from_pretrained(path_join(data_dir,'roberta.base','model.pt'),config =robert_config_question)
    model_q = RobertaForSequenceClassification_v2.from_pretrained('roberta-base',config =robert_config_question)
    
    model_q.zero_grad();
    model_q.to(device);
    torch.cuda.empty_cache()
    model_q.train();
    

     #model for answers
    #model_a = RobertaForSequenceClassification_v2.from_pretrained(path_join(data_dir,'roberta.base','model.pt'),config =robert_config_answer)
    model_a = RobertaForSequenceClassification_v2.from_pretrained('roberta-base',config =robert_config_answer)
    model_a.zero_grad();
    model_a.to(device);
    torch.cuda.empty_cache()
    model_a.train();

    #questions
    optimizer_q = torch.optim.AdamW(model_q.parameters(), lr=4e-5, eps=4e-5)
    criterion = nn.BCEWithLogitsLoss( reduction='none')
    scheduler_q = get_cosine_schedule_with_warmup(optimizer_q, num_warmup_steps=0.01, num_training_steps= epochs*len(train_loader_q)//ACCUM_STEPS)

    #answers
    optimizer_a = torch.optim.AdamW(model_a.parameters(), lr=4e-5, eps=4e-5)
    criterion = nn.BCEWithLogitsLoss( reduction='none')
    scheduler_a = get_cosine_schedule_with_warmup(optimizer_a, num_warmup_steps=0.01, num_training_steps= epochs*len(train_loader_a)//ACCUM_STEPS)

    print("Training....")

    for epoch in tqdm(range(epochs),position=0, leave=True ):        
        torch.cuda.empty_cache()

        start_time_q   = time.time()
        avg_loss_q    = train_model_questions(train_loader_q, optimizer_q, criterion, scheduler_q)
        avg_val_loss_q, score_q, preds_q = val_model_questions(valid_loader_q, val_shape=val_df.shape[0])
        elapsed_time_q = time.time() - start_time_q


        start_time_a   = time.time()
        avg_loss_a    = train_model_answers(train_loader_a, optimizer_a, criterion, scheduler_a)
        avg_val_loss_a,  score_a, preds_a = val_model_answers(valid_loader_a, val_shape=val_df.shape[0])
        elapsed_time_a = time.time() - start_time_q

        p = np.hstack((preds_q,  preds_a)) #np.hstack((preds_q,  preds_a[:,21:30]))
        o = np.hstack((outputs_valid_q[:,0:21].numpy(), outputs_valid_a[:,0:9].numpy()))   #outputs_valid_a[:,0:30].numpy()
        score = np.mean([spearmanr(o[:, i], p[:,i]).correlation for i in range(30)])

        # print(bcolors.WARNING, 'Epoch {}/{} \t Train loss Q={:.4f} \t Val loss Q={:.4f} \t Train loss A={:.4f} \t Val loss A={:.4f}  \t score={:.6f} \t time={:.2f}s'.format(
        #     epoch + 1, epochs, avg_loss_q, avg_val_loss_q,avg_loss_a, avg_val_loss_a, score, elapsed_time_a),
        # bcolors.ENDC
        # )
        print(bcolors.WARNING, 'Epoch {}/{} \t Train loss Q={:.4f} \t Val loss Q={:.4f} \t  Score Q={:.5f} \t Train loss A={:.4f} \t Val loss A={:.4f} \t Score A={:.5f}  \t score={:.6f} \t time={:.2f}s'.format(
            epoch + 1, epochs, avg_loss_q, avg_val_loss_q,  score_q, avg_loss_a, avg_val_loss_a, score_a,  score, elapsed_time_a),
        bcolors.ENDC
        )
        logging.warning('Epoch {}/{} \t Train loss Q={:.4f} \t Val loss Q={:.4f} \t  Score Q={:.5f} \t Train loss A={:.4f} \t Val loss A={:.4f} \t Score A={:.5f}  \t score={:.6f} \t time={:.2f}s'.format(
            epoch + 1, epochs, avg_loss_q, avg_val_loss_q,  score_q, avg_loss_a, avg_val_loss_a, score_a,  score, elapsed_time_a))
        
        all_rhos.append(score)

        if best_score_q < score_q:
            best_score_q = score_q
            best_e_q = epoch
            best_param_score_q = model_q.state_dict()
        else:
            i += 1
            
        if best_score_a < score_a:
            best_score_a = score_a
            best_e_a = epoch
            best_param_score_a = model_a.state_dict()
        else:
            i += 1

    best_rhos.append((best_score_q*21 + best_score_a*9)/30.0)
    torch.save(best_param_score_q,path_join('gdrive','My Drive','kaggle','submissions','roberta_seed_1029','saved_models_1029','Q_model_save_fold_{}.pt'.format(fold)))
    torch.save(best_param_score_a,path_join('gdrive','My Drive','kaggle','submissions','roberta_seed_1029','saved_models_1029','A_model_save_fold_{}.pt'.format(fold)))
        # torch.save(model_q.state_dict(),path_join('/home','jupyter', 'submissions','bert_use_features2','saved_models2','Qmodel_save_fold_{}_epoch_{}.pt'.format(fold,epoch)))
        # torch.save(model_a.state_dict(),path_join('/home','jupyter', 'submissions','bert_use_features2','saved_models2','Amodel_save_fold_{}_epoch_{}.pt'.format(fold,epoch)))


logging.info(all_rhos)
logging.warning(np.mean(best_rhos))
print("Best Rhos:",np.mean(best_rhos))
#Fold 0
#Epoch 1/5 	 Train loss Q=3.6291 	 Val loss Q=3.3467 	  Score Q=0.37285 	 Train loss A=3.0599 	 Val loss A=2.9053 	 Score A=0.26080  	 score=0.339231 
#Epoch 2/5 	 Train loss Q=3.2991 	 Val loss Q=3.2418 	  Score Q=0.41490 	 Train loss A=2.8658 	 Val loss A=2.8570 	 Score A=0.29683  	 score=0.379478 
#Epoch 3/5 	 Train loss Q=3.1689 	 Val loss Q=3.2556 	  Score Q=0.42321 	 Train loss A=2.7558 	 Val loss A=2.8924 	 Score A=0.30668  	 score=0.388249 
#Epoch 4/5 	 Train loss Q=3.0835 	 Val loss Q=3.2204 	  Score Q=0.43078 	 Train loss A=2.6517 	 Val loss A=2.8680 	 Score A=0.31594  	 score=0.396327 
#Epoch 5/5 	 Train loss Q=3.0326 	 Val loss Q=3.2294 	  Score Q=0.43011 	 Train loss A=2.5876 	 Val loss A=2.8840 	 Score A=0.31390  	 score=0.395246 

#Fold 1
#Epoch 1/5 	 Train loss Q=3.6351 	 Val loss Q=3.3773 	  Score Q=0.37860 	 Train loss A=3.0605 	 Val loss A=2.8482 	 Score A=0.26614  	 score=0.344861
#Epoch 2/5 	 Train loss Q=3.2919 	 Val loss Q=3.2975 	  Score Q=0.41038 	 Train loss A=2.8895 	 Val loss A=2.7901 	 Score A=0.29722  	 score=0.376434 
#Epoch 3/5 	 Train loss Q=3.1569 	 Val loss Q=3.2572 	  Score Q=0.42451 	 Train loss A=2.7733 	 Val loss A=2.8140 	 Score A=0.30807  	 score=0.389576 
#Epoch 4/5 	 Train loss Q=3.0682 	 Val loss Q=3.2425 	  Score Q=0.42885 	 Train loss A=2.6735 	 Val loss A=2.8094 	 Score A=0.32633  	 score=0.398096 
#Epoch 5/5 	 Train loss Q=3.0135 	 Val loss Q=3.2511 	  Score Q=0.42892 	 Train loss A=2.6079 	 Val loss A=2.8167 	 Score A=0.32991  	 score=0.399219 

#Fold 2
#Epoch 1/5 	 Train loss Q=3.6564 	 Val loss Q=3.3355 	  Score Q=0.37701 	 Train loss A=3.0213 	 Val loss A=2.9099 	 Score A=0.28440  	 score=0.349230
#Epoch 2/5 	 Train loss Q=3.3040 	 Val loss Q=3.2620 	  Score Q=0.39947 	 Train loss A=2.8633 	 Val loss A=2.8619 	 Score A=0.33732  	 score=0.380825 	 time=1161.45s 
#Epoch 3/5 	 Train loss Q=3.1668 	 Val loss Q=3.2258 	  Score Q=0.41935 	 Train loss A=2.7541 	 Val loss A=2.8580 	 Score A=0.35387  	 score=0.399709 	 time=1162.21s 
#Epoch 4/5 	 Train loss Q=3.0738 	 Val loss Q=3.2288 	  Score Q=0.42027 	 Train loss A=2.6546 	 Val loss A=2.8709 	 Score A=0.36130  	 score=0.402576 	 time=1162.43s 
#Epoch 5/5 	 Train loss Q=3.0184 	 Val loss Q=3.2307 	  Score Q=0.42141 	 Train loss A=2.5918 	 Val loss A=2.8878 	 Score A=0.35558  	 score=0.401661 

#Fold 3
#Epoch 1/5 	 Train loss Q=3.6034 	 Val loss Q=3.3458 	  Score Q=0.37910 	 Train loss A=3.0650 	 Val loss A=3.0914 	 Score A=0.26050  	 score=0.343520
#Epoch 2/5 	 Train loss Q=3.2757 	 Val loss Q=3.2794 	  Score Q=0.41307 	 Train loss A=2.8709 	 Val loss A=2.9206 	 Score A=0.30731  	 score=0.381342 
#Epoch 3/5 	 Train loss Q=3.1492 	 Val loss Q=3.2543 	  Score Q=0.42318 	 Train loss A=2.7528 	 Val loss A=2.9059 	 Score A=0.31349  	 score=0.390273 
#Epoch 4/5 	 Train loss Q=3.0567 	 Val loss Q=3.2596 	  Score Q=0.42535 	 Train loss A=2.6504 	 Val loss A=2.9141 	 Score A=0.31956  	 score=0.393616 
#Epoch 5/5 	 Train loss Q=3.0038 	 Val loss Q=3.2597 	  Score Q=0.42577 	 Train loss A=2.5788 	 Val loss A=2.9400 	 Score A=0.31793  	 score=0.393420

#Fold 4
#Epoch 1/5 	 Train loss Q=3.6454 	 Val loss Q=3.4217 	  Score Q=0.36672 	 Train loss A=3.0171 	 Val loss A=2.9213 	 Score A=0.27712  	 score=0.339841
#Epoch 2/5 	 Train loss Q=3.3042 	 Val loss Q=3.2812 	  Score Q=0.41768 	 Train loss A=2.8419 	 Val loss A=2.9026 	 Score A=0.29734  	 score=0.381582
#Epoch 3/5 	 Train loss Q=3.1631 	 Val loss Q=3.2649 	  Score Q=0.42529 	 Train loss A=2.7362 	 Val loss A=2.9021 	 Score A=0.32294  	 score=0.394586
#Epoch 4/5 	 Train loss Q=3.0724 	 Val loss Q=3.2554 	  Score Q=0.42962 	 Train loss A=2.6296 	 Val loss A=2.9024 	 Score A=0.32544  	 score=0.398366
#Epoch 5/5 	 Train loss Q=3.0194 	 Val loss Q=3.2582 	  Score Q=0.42994 	 Train loss A=2.5660 	 Val loss A=2.9175 	 Score A=0.32513  	 score=0.398495 


29it [00:00, 289.23it/s]

 Current Fold: 0 
Train and Valid Shapes are (4859, 43) (1220, 43)
 Preparing train datasets.... 


4859it [00:07, 683.01it/s]
4859it [00:11, 426.13it/s]
79it [00:00, 785.64it/s]

 Preparing Valid datasets.... 


1220it [00:01, 845.60it/s]
1220it [00:02, 478.96it/s]


...

 val_spearman-rho: 0.33413                                                                                                    
0 SpearmanrResult(correlation=0.25235146758561827, pvalue=4.4690752790481505e-19)
1 SpearmanrResult(correlation=0.31739174170374634, pvalue=8.586254969705788e-30)
2 SpearmanrResult(correlation=0.19155231153488742, pvalue=1.732933123706285e-11)
3 SpearmanrResult(correlation=0.14708396267819554, pvalue=2.6696723956422914e-07)
4 SpearmanrResult(correlation=0.2648214243375805, pvalue=6.4901660850426376e-21)
5 SpearmanrResult(correlation=0.7564403579039485, pvalue=1.5714123394152456e-225)
6 SpearmanrResult(correlation=0.2629346689395432, pvalue=1.2494987642079947e-20)
7 SpearmanrResult(correlation=0.6611664553957624, pvalue=2.5493900415023207e-153)
8 SpearmanrResult(correlation=0.15339006118558396, pvalue=7.935700920119667e-08)
 Epoch 4/5 	 Train loss Q=3.0686 	 Val loss Q=3.2498 	  Score Q=0.42720 	 Train loss A=2.6461 	 Val loss A=2.8990 	 Score A=0.33413  	 score

609it [04:53,  2.59it/s]
152it [00:23,  7.10it/s]
0it [00:00, ?it/s]

 val_spearman-rho: 0.4279                                                                                                    
0 SpearmanrResult(correlation=0.38607314449070596, pvalue=2.126776214715502e-44)
1 SpearmanrResult(correlation=0.6110192112512546, pvalue=4.727228157356428e-125)
2 SpearmanrResult(correlation=0.42218390339646195, pvalue=1.2926412458491871e-53)
3 SpearmanrResult(correlation=0.29435802312413467, pvalue=1.1337499666798355e-25)
4 SpearmanrResult(correlation=0.3119784404831541, pvalue=8.620845524575259e-29)
5 SpearmanrResult(correlation=0.4372702879190989, pvalue=8.205668329920298e-58)
6 SpearmanrResult(correlation=0.34762151658545637, pvalue=8.902572367521465e-36)
7 SpearmanrResult(correlation=0.5088604538139778, pvalue=7.067181900974485e-81)
8 SpearmanrResult(correlation=0.6158033114228009, pvalue=1.5728976678474593e-127)
9 SpearmanrResult(correlation=0.08322048627012674, pvalue=0.003726157280910468)
10 SpearmanrResult(correlation=0.4535964122126938, pvalue=1.33793

609it [04:53,  2.60it/s]
152it [00:23,  7.18it/s]
100%|██████████| 5/5 [52:48<00:00, 633.63s/it]


 val_spearman-rho: 0.33296                                                                                                    
0 SpearmanrResult(correlation=0.2491524409985504, pvalue=1.2758819222466567e-18)
1 SpearmanrResult(correlation=0.32219017834103725, pvalue=1.0675998719905727e-30)
2 SpearmanrResult(correlation=0.19060699766675535, pvalue=2.1849497266740142e-11)
3 SpearmanrResult(correlation=0.1473540680467927, pvalue=2.5370912272579485e-07)
4 SpearmanrResult(correlation=0.2618698666091672, pvalue=1.8041513538239876e-20)
5 SpearmanrResult(correlation=0.7533523824700271, pvalue=1.1178693970335613e-222)
6 SpearmanrResult(correlation=0.26492486440370766, pvalue=6.2602698348793954e-21)
7 SpearmanrResult(correlation=0.6608123060949125, pvalue=4.21995576174498e-153)
8 SpearmanrResult(correlation=0.14637752738072218, pvalue=3.048792109733305e-07)
 Epoch 5/5 	 Train loss Q=3.0126 	 Val loss Q=3.2517 	  Score Q=0.42790 	 Train loss A=2.5796 	 Val loss A=2.9262 	 Score A=0.33296  	 score=

In [0]:
#seed 101 = Best Rhos: 0.40001835074053727
#fold 0
#Epoch 1/5 	 Train loss Q=3.6465 	 Val loss Q=3.3752 	  Score Q=0.37313 	 Train loss A=3.0130 	 Val loss A=2.9009 	 Score A=0.30204  	 score=0.351800 
#Epoch 2/5 	 Train loss Q=3.3065 	 Val loss Q=3.2708 	  Score Q=0.41303 	 Train loss A=2.8496 	 Val loss A=2.8537 	 Score A=0.32592  	 score=0.386899 
#Epoch 3/5 	 Train loss Q=3.1729 	 Val loss Q=3.2182 	  Score Q=0.42667 	 Train loss A=2.7434 	 Val loss A=2.8512 	 Score A=0.33558  	 score=0.399343 
#Epoch 4/5 	 Train loss Q=3.0805 	 Val loss Q=3.2230 	  Score Q=0.42826 	 Train loss A=2.6389 	 Val loss A=2.8626 	 Score A=0.33821  	 score=0.401243 
#Epoch 5/5 	 Train loss Q=3.0243 	 Val loss Q=3.2256 	  Score Q=0.42841 	 Train loss A=2.5809 	 Val loss A=2.8796 	 Score A=0.33659  	 score=0.400865

#fold 1
#Epoch 1/5 	 Train loss Q=3.6608 	 Val loss Q=3.4193 	  Score Q=0.37832 	 Train loss A=3.0450 	 Val loss A=2.8930 	 Score A=0.27187  	 score=0.346386
#Epoch 2/5 	 Train loss Q=3.3020 	 Val loss Q=3.2908 	  Score Q=0.41636 	 Train loss A=2.8527 	 Val loss A=2.8901 	 Score A=0.29938  	 score=0.381267 
#Epoch 3/5 	 Train loss Q=3.1659 	 Val loss Q=3.2635 	  Score Q=0.41941 	 Train loss A=2.7586 	 Val loss A=2.8404 	 Score A=0.32161  	 score=0.390069 
#Epoch 4/5 	 Train loss Q=3.0751 	 Val loss Q=3.2547 	  Score Q=0.42925 	 Train loss A=2.6593 	 Val loss A=2.8685 	 Score A=0.32359  	 score=0.397552
#Epoch 5/5 	 Train loss Q=3.0206 	 Val loss Q=3.2569 	  Score Q=0.42965 	 Train loss A=2.5987 	 Val loss A=2.8739 	 Score A=0.32882  	 score=0.399401

#fold 2
#Epoch 1/5 	 Train loss Q=3.6485 	 Val loss Q=3.3970 	  Score Q=0.37134 	 Train loss A=3.0209 	 Val loss A=2.9240 	 Score A=0.26258  	 score=0.338716
#Epoch 2/5 	 Train loss Q=3.3063 	 Val loss Q=3.2726 	  Score Q=0.40817 	 Train loss A=2.8490 	 Val loss A=2.8695 	 Score A=0.29184  	 score=0.373267 
#Epoch 3/5 	 Train loss Q=3.1726 	 Val loss Q=3.2544 	  Score Q=0.41980 	 Train loss A=2.7474 	 Val loss A=2.8555 	 Score A=0.31695  	 score=0.388946 
#Epoch 4/5 	 Train loss Q=3.0783 	 Val loss Q=3.2402 	  Score Q=0.42286 	 Train loss A=2.6486 	 Val loss A=2.8895 	 Score A=0.32781  	 score=0.394344
#Epoch 5/5 	 Train loss Q=3.0287 	 Val loss Q=3.2486 	  Score Q=0.42310 	 Train loss A=2.5868 	 Val loss A=2.9125 	 Score A=0.32722  	 score=0.394332 

#fold 3
#Epoch 1/5 	 Train loss Q=3.6445 	 Val loss Q=3.3337 	  Score Q=0.38704 	 Train loss A=3.0510 	 Val loss A=2.9127 	 Score A=0.24856  	 score=0.345500 
#Epoch 2/5 	 Train loss Q=3.3088 	 Val loss Q=3.2371 	  Score Q=0.42216 	 Train loss A=2.8674 	 Val loss A=2.8869 	 Score A=0.30683  	 score=0.387561 
#poch 3/5 	 Train loss Q=3.1785 	 Val loss Q=3.2187 	  Score Q=0.43165 	 Train loss A=2.7655 	 Val loss A=2.8162 	 Score A=0.34497  	 score=0.405645 
#Epoch 4/5 	 Train loss Q=3.0848 	 Val loss Q=3.2250 	  Score Q=0.43497 	 Train loss A=2.6583 	 Val loss A=2.8393 	 Score A=0.35230  	 score=0.410167 
#Epoch 5/5 	 Train loss Q=3.0318 	 Val loss Q=3.2208 	  Score Q=0.43622 	 Train loss A=2.5872 	 Val loss A=2.8599 	 Score A=0.35194  	 score=0.410933 

#fold 4
#Epoch 1/5 	 Train loss Q=3.6318 	 Val loss Q=3.3908 	  Score Q=0.37632 	 Train loss A=3.0399 	 Val loss A=2.8916 	 Score A=0.28689  	 score=0.349489 
#Epoch 2/5 	 Train loss Q=3.2875 	 Val loss Q=3.3016 	  Score Q=0.41263 	 Train loss A=2.8559 	 Val loss A=2.8728 	 Score A=0.29738  	 score=0.378053
#Epoch 3/5 	 Train loss Q=3.1647 	 Val loss Q=3.2512 	  Score Q=0.42451 	 Train loss A=2.7322 	 Val loss A=2.9082 	 Score A=0.30192  	 score=0.387729
#Epoch 4/5 	 Train loss Q=3.0753 	 Val loss Q=3.2565 	  Score Q=0.42699 	 Train loss A=2.6290 	 Val loss A=2.9158 	 Score A=0.31597  	 score=0.393682
#Epoch 5/5 	 Train loss Q=3.0221 	 Val loss Q=3.2533 	  Score Q=0.42714 	 Train loss A=2.5673 	 Val loss A=2.9345 	 Score A=0.31109  	 score=0.392324 

# Seed 42
#Fold 0 
#Epoch 1/5 	 Train loss Q=3.6090 	 Val loss Q=3.3769 	  Score Q=0.38789 	 Train loss A=3.0358 	 Val loss A=2.9281 	 Score A=0.25481  	 score=0.347965 
#Epoch 2/5 	 Train loss Q=3.2809 	 Val loss Q=3.2712 	  Score Q=0.41401 	 Train loss A=2.8714 	 Val loss A=2.8501 	 Score A=0.30459  	 score=0.381183 
#Epoch 3/5 	 Train loss Q=3.1597 	 Val loss Q=3.2413 	  Score Q=0.42741 	 Train loss A=2.7692 	 Val loss A=2.8257 	 Score A=0.32929  	 score=0.397977
#Epoch 4/5 	 Train loss Q=3.0639 	 Val loss Q=3.2401 	  Score Q=0.42968 	 Train loss A=2.6711 	 Val loss A=2.8494 	 Score A=0.33965  	 score=0.402669 
#Epoch 5/5 	 Train loss Q=3.0106 	 Val loss Q=3.2400 	  Score Q=0.42994 	 Train loss A=2.6066 	 Val loss A=2.8563 	 Score A=0.34197  	 score=0.403548 

#Fold 1 (seems like I ran fold 1 first!!)
#Epoch 1/5 	 Train loss Q=3.6729 	 Val loss Q=3.4193 	  Score Q=0.36193 	 Train loss A=3.0376 	 Val loss A=2.8714 	 Score A=0.25758  	 score=0.330624 
#Epoch 2/5 	 Train loss Q=3.3151 	 Val loss Q=3.2850 	  Score Q=0.40845 	 Train loss A=2.8580 	 Val loss A=2.8161 	 Score A=0.31599  	 score=0.380709 
#Epoch 3/5 	 Train loss Q=3.1757 	 Val loss Q=3.2270 	  Score Q=0.42401 	 Train loss A=2.7472 	 Val loss A=2.8132 	 Score A=0.33506  	 score=0.397328 
#Epoch 4/5 	 Train loss Q=3.0787 	 Val loss Q=3.2295 	  Score Q=0.42494 	 Train loss A=2.6447 	 Val loss A=2.8384 	 Score A=0.34775  	 score=0.401781
#Epoch 5/5 	 Train loss Q=3.0240 	 Val loss Q=3.2328 	  Score Q=0.42581 	 Train loss A=2.5796 	 Val loss A=2.8537 	 Score A=0.34537  	 score=0.401677 

#Fold 2
#Epoch 1/5 	 Train loss Q=3.6480 	 Val loss Q=3.3509 	  Score Q=0.39388 	 Train loss A=3.0346 	 Val loss A=2.9463 	 Score A=0.25937  	 score=0.353528 
#Epoch 2/5 	 Train loss Q=3.3015 	 Val loss Q=3.2849 	  Score Q=0.41583 	 Train loss A=2.8625 	 Val loss A=2.8586 	 Score A=0.30475  	 score=0.382504 


In [0]:
all_rhos

[0.35180030951654445,
 0.38689907071757834,
 0.39934297582903827,
 0.4012427167083823,
 0.40086516255312965,
 0.3463861853241752,
 0.38126744429415166,
 0.3900687706930823,
 0.397551690809272,
 0.3994012901122473,
 0.3387159686838434,
 0.3732671378480203,
 0.3889463665524084,
 0.3943438783009059,
 0.3943321358639862,
 0.3455003449456339,
 0.38756051102503614,
 0.4056445897249139,
 0.4101674718980629,
 0.41093260828006717,
 0.34948943900545043,
 0.37805251295942816,
 0.38772916493152815,
 0.3936821972163829,
 0.3923236366304047]

In [0]:
np.mean(all_rhos)

0.38422054321694693